In [20]:
import pandas as pd

# 파일 경로 설정 및 데이터 로드
file_path_df = 'train_ver5.csv'  # 실제 경로로 교체하세요
df = pd.read_csv(file_path_df)

file_path_labels = 'train_label.csv'  # 실제 경로로 교체하세요
labels_df = pd.read_csv(file_path_labels)

file_path_val_df = 'val_ver4.csv'  # 실제 경로로 교체하세요
val_df = pd.read_csv(file_path_val_df)

file_path_val_labels_df = 'val_label.csv'  # 실제 경로로 교체하세요
val_labels_df = pd.read_csv(file_path_val_labels_df)

new_df = pd.concat([df, val_df], ignore_index=True)

new_labels_df = pd.concat([labels_df, val_labels_df], ignore_index=True)

# 새로운 CSV 파일로 저장
new_df.to_csv('new_df.csv', index=False)
new_labels_df.to_csv('new_label.csv', index=False)



In [51]:
# import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
import torch.nn as nn
from torch_geometric.nn import GATConv
import torch.nn.functional as F
from torch.nn import ModuleList
import random
import torch_geometric

# 하이퍼파라미터 설정
learning_rate = 0.001
d_o = 0.8
epoch = 500
percent = 20

# 시드 고정 함수
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch_geometric.seed_everything(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 시드 고정
set_seed(42)

# 데이터 로드
file_path = 'new_df.csv'  # 실제 경로로 교체하세요
df = pd.read_csv(file_path)

file_path = 'new_label.csv'  # 실제 경로로 교체하세요
labels_df = pd.read_csv(file_path)

# 라벨 텐서 생성
y = torch.tensor(labels_df['label'].values, dtype=torch.long)

# 트레인과 밸리데이션 데이터 나누기
train_df = df.iloc[:9704]  # 첫 9704개의 행
val_df = df.iloc[9704:]  # 9705번째 행부터 끝까지

train_labels = y[:9704]
val_labels = y[9704:]

# 트레인 + 밸리데이션 데이터 합치기 (전체 데이터)
all_df = pd.concat([train_df, val_df], axis=0)
all_labels = torch.cat([train_labels, val_labels], dim=0)

# 피어슨 상관계수로 엣지 리스트 생성 (전체 데이터에 대해)
pearson_corr = all_df.corr(method='pearson')
corr_array = pearson_corr.values
np.fill_diagonal(corr_array, np.nan)
flattened_corr = corr_array.flatten()
valid_corr = flattened_corr[~np.isnan(flattened_corr)]
top_percent_value = np.percentile(valid_corr, percent)
edge_indices = np.argwhere(corr_array >= top_percent_value)
edge_list = [[int(edge[0]), int(edge[1])] for edge in edge_indices]
edge_tensor = torch.tensor(edge_list, dtype=torch.long).T

# PyTorch 텐서로 변환
all_tensor_data = torch.tensor(all_df.values, dtype=torch.float32)

# 트레인/밸리데이션 마스크 생성
train_mask = torch.zeros(all_df.shape[0], dtype=torch.bool)
train_mask[:train_df.shape[0]] = True  # 첫 9704개의 노드는 트레인 데이터

val_mask = torch.zeros(all_df.shape[0], dtype=torch.bool)
val_mask[train_df.shape[0]:] = True  # 나머지는 발리데이션 데이터

# 데이터 객체 생성 (전체 데이터)
all_data = Data(x=all_tensor_data, edge_index=edge_tensor, y=all_labels, train_mask=train_mask, val_mask=val_mask)

# GPU 사용 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
all_data = all_data.to(device)
num_classes = len(all_labels.unique())

# GAT 모델 정의
class GAT(nn.Module):
    def __init__(self):
        super().__init__()
        self.convs = ModuleList([
            GATConv(all_data.num_node_features, 8, heads=8, dropout=d_o),
            GATConv(8 * 8, num_classes, heads=1, concat=False, dropout=d_o),
        ])

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i != len(self.convs) - 1:
                x = F.relu(x)
                x = F.dropout(x, p=d_o, training=self.training)
        return F.log_softmax(x, dim=-1)

# 학습 함수
def train_node_classifier(model, graph, optimizer, criterion, n_epochs=epoch):
    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph)
        loss = criterion(out[graph.train_mask], graph.y[graph.train_mask])  # 트레인 마스크에 해당하는 노드만 학습
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f'Epoch: {epoch:03d}, Train Loss: {loss:.3f}')

    return model

# 모델 학습 및 평가
gcn = GAT().to(device)
optimizer_gcn = torch.optim.Adam(gcn.parameters(), lr=learning_rate, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()
gcn = train_node_classifier(gcn, all_data, optimizer_gcn, criterion)

# Val 데이터에 대한 예측 및 정확도 계산 (전체 데이터로부터 마스크 사용)
gcn.eval()
with torch.no_grad():
    val_pred = gcn(all_data).argmax(dim=1)
    correct_val = (val_pred[all_data.val_mask] == all_data.y[all_data.val_mask]).sum()
    val_accuracy = int(correct_val) / all_data.val_mask.sum().item()

# 결과 출력
print(f'Val 데이터셋에 대한 예측 정확도: {val_accuracy:.3f}')
print(f'Val 데이터셋 실제 레이블: {all_data.y[all_data.val_mask]}')
print(f'Val 데이터셋 예측된 레이블: {val_pred[all_data.val_mask]}')


Epoch: 100, Train Loss: 1.049
Epoch: 200, Train Loss: 1.009
Epoch: 300, Train Loss: 0.975
Epoch: 400, Train Loss: 0.948
Epoch: 500, Train Loss: 0.927
Val 데이터셋에 대한 예측 정확도: 0.789
Val 데이터셋 실제 레이블: tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0')
Val 데이터셋 예측된 레이블: tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0')
